In [1]:
import pandas as pd
import datetime
from datetime import date, time, timedelta
import itertools
import time
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Global variables (CAPS)

battery_power = 300
battery_cap = 580
charge_eff = 90
discharge_eff = 90
mlf = 0.991

In [4]:
def raw_power(charge_forecast, discharge_forecast, opening_cap):
    if charge_forecast == 1 and discharge_forecast == 0:
        return -min(battery_power, (battery_cap - opening_cap)/(charge_eff/100)*2)
    elif charge_forecast == 0 and discharge_forecast == 1:
        return min(battery_power, (opening_cap/(discharge_eff/100))*2 )
    else:
        return 0

In [5]:
def market_dispatch(raw_power):
    if raw_power < 0:
        return (raw_power/2)
    elif raw_power > 0:
        return (raw_power/2)*discharge_eff/100
    else:
        return 0

In [6]:
def market_revenue(market_dispatch,spot_price):
    if market_dispatch < 0:
        return market_dispatch*spot_price*(1/mlf)
    elif market_dispatch > 0:
        return market_dispatch*spot_price*mlf
    else:
        return 0

In [7]:
def closing_capacity(market_dispatch, opening_cap):
    if market_dispatch < 0:
        x = opening_cap - market_dispatch * (charge_eff/100)
        return (max(0, min(round(x), battery_cap)))
    elif market_dispatch > 0:
        x = opening_cap - market_dispatch * (100/discharge_eff)
        return (max(0, min(round(x), battery_cap)))
    else:
        x = opening_cap - market_dispatch * (100/discharge_eff)
        return (max(0, min(round(x), battery_cap)))

In [16]:
def quantile_prices(index):
    past_quantile = 0
    future_quantile = 0
    if index < 10:   # if number of past prices less than 10
        past_quantile = subsample.iloc[0:index]['Spot Price'].quantile(0.80)
        future_quantile = subsample.iloc[index+1:index + 11]['Spot Price'].quantile(0.20)
    elif index >= (len(subsample) - 10):   # if number of future prices more than 10
        past_quantile = subsample.iloc[index - 10:index]['Spot Price'].quantile(0.80)
        future_quantile = subsample.iloc[index+1:len(subsample)]['Spot Price'].quantile(0.20)
    else:
        past_quantile = subsample.iloc[index - 10:index]['Spot Price'].quantile(0.80)
        future_quantile = subsample.iloc[index+1:index + 11]['Spot Price'].quantile(0.20)
    return past_quantile, future_quantile

In [35]:
def charging_condition(index, current_price, past_quantile, future_quantile):
    ''' Determine whether current period is forcasted to charge or discharge '''
    
    lower_threshold = 2
    upper_threshold = 10
    
    current_past_diff = current_price - past_quantile
    current_future_diff = future_quantile - current_price    
    
    # Discharge conditions
    if (current_price > past_quantile) and (current_price > future_quantile):
        if current_past_diff >= upper_threshold:
            subsample["Discharge Forecast"].values[index] = 1
    
    # Charge conditions
    elif (current_price < past_quantile) and (current_price < future_quantile):
        if current_future_diff >= lower_threshold:
            subsample["Charge Forecast"].values[index] = 1
            
    elif (current_price > past_quantile) and (current_price < future_quantile):
        if current_past_diff >= upper_threshold and current_future_diff >= lower_threshold:
            if max(current_past_diff, current_future_diff) == current_past_diff:
                subsample["Discharge Forecast"].values[index] = 1
            else:
                subsample["Charge Forecast"].values[index] = 1
        elif current_past_diff >= upper_threshold and current_future_diff < lower_threshold:
            subsample["Discharge Forecast"].values[index] = 1
        elif current_future_diff >= lower_threshold and current_past_diff < upper_threshold:
            subsample["Charge Forecast"].values[index] = 1

In [10]:
df = pd.read_excel("../../data/market_data.xlsx") 

In [11]:
vic_spotprice = df.filter(items=['Time (UTC+10)', 'Regions VIC Trading Price ($/MWh)'])
vic_spotprice = vic_spotprice.rename(columns={'Regions VIC Trading Price ($/MWh)': 'Spot Price', 'Time (UTC+10)': 'Time'})
# Change column type to datetime type
vic_spotprice['Time'] = pd.to_datetime(vic_spotprice['Time'])

In [41]:
# Subsample first 3 days
subsample = vic_spotprice.loc[vic_spotprice['Time'].dt.date <= date(2020, 12, 31)]
# subsample

In [42]:
# Create new columns

subsample["Past Quantile"] = 0.0
subsample["Future Quantile"] = 0.0
subsample["Charge Forecast"] = 0
subsample["Discharge Forecast"] = 0
subsample["Charge Forecast"] = 0
subsample["Discharge Forecast"] = 0
subsample["Raw Power"] = 0
subsample["Market Dispatch"] = 0
subsample["Market Revenue"] = 0
subsample["Opening Capacity"] = 0
subsample["Closing Capacity"] = 0

In [43]:
%%time

for index, row in subsample.iterrows():
    past_quantile_prices, future_quantile_prices = quantile_prices(index)
    subsample["Past Quantile"].values[index] = past_quantile_prices
    subsample["Future Quantile"].values[index] = future_quantile_prices
    
    current_price = subsample.iloc[index]["Spot Price"]
    past_quantile = subsample.iloc[index]["Past Quantile"]
    future_quantile = subsample.iloc[index]["Future Quantile"]

    charging_condition(index, current_price, past_quantile, future_quantile)
    
    if index != 0:
        subsample["Opening Capacity"].values[index] = subsample["Closing Capacity"].values[index-1]
    
    subsample["Raw Power"].values[index] = raw_power(subsample["Charge Forecast"].values[index], subsample["Discharge Forecast"].values[index], subsample["Opening Capacity"].values[index])
    subsample["Market Dispatch"].values[index] = market_dispatch(subsample["Raw Power"].values[index])
    subsample["Market Revenue"].values[index] = market_revenue(subsample["Market Dispatch"].values[index], subsample["Spot Price"].values[index])
    subsample["Closing Capacity"].values[index] = closing_capacity(subsample["Market Dispatch"].values[index], subsample["Opening Capacity"].values[index])

CPU times: user 1min 24s, sys: 19.7 ms, total: 1min 24s
Wall time: 1min 24s


In [44]:
# Current > Past  --> Discharge
# Current < Future --> Charge

# subsample.tail(50)
sum(subsample["Market Revenue"])

65006512

In [40]:
subsample.head(50)

,Time,Spot Price,Past Quantile,Future Quantile,Charge Forecast,Discharge Forecast,Raw Power,Market Dispatch,Market Revenue,Opening Capacity,Closing Capacity
0,2018-01-01 00:00:00,90.43,NaN,65.482,0,0,0,0,0,0,0
1,2018-01-01 00:30:00,92.46,90.430,65.228,0,0,0,0,0,0,0
2,2018-01-01 01:00:00,87.62,92.054,64.332,0,0,0,0,0,0,0
3,2018-01-01 01:30:00,73.08,91.648,64.332,0,0,0,0,0,0,0
4,2018-01-01 02:00:00,70.18,91.242,64.332,0,0,0,0,0,0,0
5,2018-01-01 02:30:00,67.43,90.836,64.332,0,0,0,0,0,0,0
6,2018-01-01 03:00:00,66.31,90.430,64.332,0,0,0,0,0,0,0
7,2018-01-01 03:30:00,67.72,89.868,64.332,0,0,0,0,0,0,0
8,2018-01-01 04:00:00,65.50,89.306,64.332,0,0,0,0,0,0,0
9,2018-01-01 04:30:00,64.50,88.744,64.468,0,0,0,0,0,0,0
